In [1]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import pypyodbc
import csv
import os
import tempfile

# Download PLUTO data
## (For now, skip this since the Assessment Data contains better detail)
B-B-L is primary key

In [2]:
"""
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_15v1.zip')
pluto_zip = zipfile.ZipFile(StringIO.StringIO(r.content))
"""

"\nr = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_15v1.zip')\npluto_zip = zipfile.ZipFile(StringIO.StringIO(r.content))\n"

In [3]:
"""
list_pluto_files = [('Mn.csv',1),('BX.csv',2),('BK.csv',3),('QN.csv',4),('SI.csv',5)]
pluto = pd.DataFrame()
for filename, boroid in list_pluto_files:
    borodata = pd.read_csv(pluto_zip.open(filename))
    # Need to add a boroid as an integer, for joining to other sources
    borodata['boroid'] = boroid
    pluto = pluto.append(borodata)
"""

"\nlist_pluto_files = [('Mn.csv',1),('BX.csv',2),('BK.csv',3),('QN.csv',4),('SI.csv',5)]\npluto = pd.DataFrame()\nfor filename, boroid in list_pluto_files:\n    borodata = pd.read_csv(pluto_zip.open(filename))\n    # Need to add a boroid as an integer, for joining to other sources\n    borodata['boroid'] = boroid\n    pluto = pluto.append(borodata)\n"

In [4]:
# pluto.shape

# Read the Assessment rolls from DOF website zips
B-B-L-E is primary key

#### First for Tax Class 1 data

In [2]:
tc1_source = 'http://www1.nyc.gov/assets/finance/downloads/tar/tc1_16.zip'
r1 = requests.get(tc1_source)
tc1_zip = zipfile.ZipFile(StringIO.StringIO(r1.content))
tc1 = tc1_zip.read('tc1.mdb')
tc1_zip.close()
print(len(tc1))

825106432


#### dump the extracted Access Database into a tempfile

In [3]:
temp1 = tempfile.NamedTemporaryFile(delete=False)
temp1.write(tc1)
temp1.seek(0)
temp1.close()

#### Connect to it to query some data and dump to a dataframe

In [4]:
# connect to db
conn_str = 'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + temp1.name
con = pypyodbc.connect(conn_str)
cur = con.cursor()

In [5]:
# run a query and get the results 
SQL = 'SELECT BBLE, BORO, BLOCK, LOT, EASE, APTNO, GR_SQFT, CUR_FV_T, CURAVT, CUREXT, CURAVT_A, CUREXT_A, FN_AVT, FN_EXT, TXCL FROM tc1;' # your query goes here
rows = cur.execute(SQL).fetchall()
cur.close()
con.close()

In [6]:
assessments_tc1 = pd.DataFrame.from_records(rows,columns=('BBLE','BORO','BLOCK','LOT','EASE','APTNO','GR_SQFT','CUR_FV_T','CURAVT','CUREXT','CURAVT_A','CUREXT_A','FN_AVT','FN_EXT','TXCL'))
assessments_tc1.shape

(708676, 15)

#### And then for Tax Class 2,3,4 data

In [7]:
tc234_source = 'http://www1.nyc.gov/assets/finance/downloads/tar/tc234_16.zip'
r234 = requests.get(tc234_source)
tc234_zip = zipfile.ZipFile(StringIO.StringIO(r234.content))
tc234 = tc234_zip.read('tc234.mdb')
tc234_zip.close()
print(len(tc234))

505520128


In [8]:
temp234 = tempfile.NamedTemporaryFile(delete=False)
temp234.write(tc234)
temp234.seek(0)
temp234.close()

In [9]:
# connect to db
conn_str = 'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + temp234.name
con = pypyodbc.connect(conn_str)
cur = con.cursor()
# run a query and get the results 
SQL = 'SELECT BBLE, BORO, BLOCK, LOT, EASE, APTNO, GR_SQFT, CUR_FV_T, CURAVT, CUREXT, CURAVT_A, CUREXT_A, FN_AVT, FN_EXT, TXCL FROM tc234;' # your query goes here
rows = cur.execute(SQL).fetchall()
cur.close()
con.close()
assessments_tc234 = pd.DataFrame.from_records(rows,columns=('BBLE','BORO','BLOCK','LOT','EASE','APTNO','GR_SQFT','CUR_FV_T','CURAVT','CUREXT','CURAVT_A','CUREXT_A','FN_AVT','FN_EXT','TXCL'))
assessments_tc234.shape

(387571, 15)

In [10]:
assessments = pd.concat([assessments_tc1,assessments_tc234])
del assessments_tc1
del assessments_tc234
assessments.query('BORO == 1 & BLOCK == 1114 & LOT == 1452')

,BBLE,BORO,BLOCK,LOT,EASE,APTNO,GR_SQFT,CUR_FV_T,CURAVT,CUREXT,CURAVT_A,CUREXT_A,FN_AVT,FN_EXT,TXCL
70223,1011141452,1,1114,1452,None,37A,3105,1142765,492773,221726,514244,234609,503867,152255,2


#### don't forget to delete the temp files!

In [11]:
os.remove(temp1.name)
os.remove(temp234.name)

In [12]:
assessments.to_csv('dof_assessments_merged.csv')

# Get the sales data from DOF site (xls)
No primary key! (Cannot uniquely identify records using any combination of fields).  Will be difficult to join this with property data.

In [16]:
sales_detail = pd.DataFrame()
list_sales_detail = ['http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2014/2014_manhattan.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2014/2014_bronx.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2014/2014_brooklyn.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2014/2014_queens.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2014/2014_statenisland.xls']
older_list =       ['http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2013/2013_manhattan.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2013/2013_bronx.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2013/2013_brooklyn.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2013/2013_queens.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2013/2013_statenisland.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2012/2012_manhattan.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2012/2012_bronx.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2012/2012_brooklyn.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2012/2012_queens.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2012/2012_statenisland.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2011/2011_manhattan.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2011/2011_bronx.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2011/2011_brooklyn.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2011/2011_queens.xls',
                    'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2011/2011_statenisland.xls']
even_older_list = ['http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2010/2010_manhattan.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2010/2010_bronx.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2010/2010_brooklyn.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2010/2010_queens.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2010/2010_statenisland.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2009_manhattan.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2009_bronx.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2009_brooklyn.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2009_queens.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2009_statenisland.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/09pdf/rolling_sales/sales_2008_manhattan.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/09pdf/rolling_sales/sales_2008_bronx.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/09pdf/rolling_sales/sales_2008_brooklyn.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/09pdf/rolling_sales/sales_2008_queens.xls',
             'http://www1.nyc.gov/assets/finance/downloads/pdf/09pdf/rolling_sales/sales_2008_statenisland.xls']

In [17]:
for url in list_sales_detail:
    xl_data = pd.read_excel(url, header=4)
    for c in xl_data.columns:
        c.rstrip('\n')
        xl_data.rename(columns={c: c.rstrip('\n')}, inplace=True)
    sales_detail = sales_detail.append(xl_data)

In [18]:
print(sales_detail.shape)
print(len(sales_detail.groupby(['BOROUGH','BLOCK','LOT','ADDRESS','APARTMENT NUMBER','SALE PRICE','SALE DATE']).size()))

(88687, 21)
86330


In [19]:
# check how many rows result from an inner join
sales_detail.merge(assessments,how='inner',left_on=['BOROUGH','BLOCK','LOT'],right_on=['BORO','BLOCK','LOT'],left_index=True).shape

(89354, 32)

In [20]:
sales_detail.to_csv('dof_sales_merged.csv')

# Get the Multiple Dwelling Registrations
B-B-L and Building ID is primary key

In [22]:
client = Socrata("data.cityofnewyork.us", "JbH87Ju4WIAzc35ZUpMxLxFyM")
mdr = client.get("39m2-a2pz", content_type="csv", limit=50000)

In [23]:
mdrdf =pd.DataFrame(mdr)
mdrdf.columns = mdrdf.iloc[0]
mdrdf = mdrdf.reindex(mdrdf.index.drop(0))
mdrdf = mdrdf.convert_objects(convert_numeric=True)
mdrdf.head()

C:\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,bin,block,boro,boroid,buildingid,communityboard,highhousenumber,housenumber,lastregistrationdate,lot,lowhousenumber,registrationenddate,registrationid,streetcode,streetname,zip
1,2065040,4873,BRONX,2,70619,12,1067,1067,2011-10-15T00:00:00.000,106,1067,2012-10-01T00:00:00.000,222287,28410,EAST 226 STREET,10466
2,4080937,3392,QUEENS,4,661272,5,1924,1924,2015-11-04T00:00:00.000,29,1924,2016-09-01T00:00:00.000,413124,48290,GROVE STREET,11385
3,4042948,1731,QUEENS,4,508388,3,NaN,NaN,2015-11-04T00:00:00.000,42,NaN,2016-09-01T00:00:00.000,410490,19040,97 STREET,11368
4,4544783,1321,QUEENS,4,692605,2,NaN,NaN,2015-11-25T00:00:00.000,61,NaN,2016-09-01T00:00:00.000,916538,61090,ROOSEVELT AVENUE,11377
5,4280961,13069,QUEENS,4,552480,13,NaN,NaN,2015-11-27T00:00:00.000,21,NaN,2016-09-01T00:00:00.000,916385,22820,143 AVENUE,11413


# Get the list of properties receiving 421-a exemption from DOF site (xls)
B-B-L is primary key

In [6]:
exempt_421a = pd.DataFrame()
list_url_421a = ['http://www1.nyc.gov/assets/finance/downloads/pdf/421a/1516/421a_1516_manhattan.xls',
                 'http://www1.nyc.gov/assets/finance/downloads/pdf/421a/1516/421a_1516_bronx.xls',
                 'http://www1.nyc.gov/assets/finance/downloads/pdf/421a/1516/421a_1516_brooklyn.xls',
                 'http://www1.nyc.gov/assets/finance/downloads/pdf/421a/1516/421a_1516_queens.xls',
                 'http://www1.nyc.gov/assets/finance/downloads/pdf/421a/1516/421a_1516_statenisland.xls']

In [7]:
for url in list_url_421a:
    exempt_421a = exempt_421a.append(pd.read_excel(url, header=3))
exempt_421a.shape

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT
0,3,BATH BEACH,02 2-FAMILY,1,6400,42,B3,8742 BAY 16 STREET,11214,2,0,2,3867,1452,1899
1,3,BATH BEACH,03 3-FAMILY,1,6364,62,C0,46 BAY 14 STREET,11214,3,0,3,2167,4568,2000
2,3,BATH BEACH,03 3-FAMILY,1,6364,63,C0,46A BAY 14 STREET,11214,3,0,3,2167,4568,2000
3,3,BATH BEACH,03 3-FAMILY,1,6366,20,C0,8651 17 AVENUE,11214,3,0,3,2803,4000,2003
4,3,BATH BEACH,03 3-FAMILY,1,6366,22,C0,8649 17 AVENUE,11214,3,0,3,2030,4200,2004


# Get the RPIE Non-Compliance List

In [24]:
list_rpienc_files = [('http://www1.nyc.gov/assets/finance/downloads/pdf/rpie/non-filers/non-filers_manhattan.xlsx',1),
                    ('http://www1.nyc.gov/assets/finance/downloads/pdf/rpie/non-filers/non-filers_bronx.xlsx',2),
                    ('http://www1.nyc.gov/assets/finance/downloads/pdf/rpie/non-filers/non-filers_brooklyn.xlsx',3),
                    ('http://www1.nyc.gov/assets/finance/downloads/pdf/rpie/non-filers/non-filers_queens.xlsx',4),
                    ('http://www1.nyc.gov/assets/finance/downloads/pdf/rpie/non-filers/non-filers_statenisland.xlsx',5)]

In [25]:
rpienc = pd.DataFrame()
for filename, boroid in list_rpienc_files:
    borodata = pd.read_excel(filename, header=1)
    # Need to add a boroid as an integer, for joining to other sources
    borodata['boroid'] = boroid
    rpienc = rpienc.append(borodata)

In [26]:
rpienc.shape

(15937, 11)